# Результаты обучения U-Net на DeepRockSR-2D

In [3]:
from pathlib import Path
from PIL import Image, ImageDraw, ImageFont
import numpy as np
import matplotlib.pyplot as plt
import re

from IPython.display import display

# ==== ПУТИ ====
# корень с предсказаниями U-Net x2, внутри ожидаются подпапки пород:
#   ROOT / "carbonate2D", ROOT / "sandstone2D", ...
ROOT = Path("C:/Users/Вячеслав/Documents/superresolution/preds/preds_test12")

# папка, куда всё складываем
OUT_DIR = Path("../../deeprock_x2_report")
OUT_DIR.mkdir(parents=True, exist_ok=True)

# список пород (если отличаются — поправишь)
MATERIALS = ["carbonate2D", "sandstone2D", "coal2D"]

# сколько троек (LR/HR/PRED) на породу использовать в отчёте
MAX_TRIPLETS_PER_MAT = 5   # можно увеличить или убрать лимит (None)

# во сколько раз HR крупнее LR
# (если у тебя не x2, можно вычислять фактор по размерам внутри кода)
UPSCALE_FACTOR = 2

# ==== ПАРАМЕТРЫ КРОПА ====
# По умолчанию: квадрат из центра с небольшим смещением
GLOBAL_BOX_SIZE = (100, 100)   # (w, h) «небольшой» box в координатах HR
GLOBAL_BOX_OFFSET = (0, 0)     # смещение от центра (dx, dy) в HR

# Можно задать абсолютные боксы по породам
# (если False — используется центр + GLOBAL_BOX_SIZE/GLOBAL_BOX_OFFSET)
USE_ABS_BOX = {
    "default": False,
    "carbonate2D": True,
    "sandstone2D": True,
    "coal2D": True,
}

ABS_BOX = {
    # формат: (x0, y0, x1, y1) в координатах HR
    "default": (50, 50, 150, 150),
    # примеры, при желании можно задать разные:
    "carbonate2D": (80, 80, 180, 180),
    "coal2D": (80, 80, 180, 180),
    "sandstone2D": (80, 80, 180, 180),
}

# ==== ВИЗУАЛЬНЫЕ ПАРАМЕТРЫ ДЛЯ МОНТАЖЕЙ ====
PADDING = 8         # поле внутри тайла
GUTTER = 12         # расстояние между тайлами
FONT_SIZE = 72
UPSCALE_TILE = 4    # во сколько раз масштабировать картинку в тайле (для просмотра)
RECT_WIDTH = 3      # толщина красной рамки на HR

# Шрифт
try:
    FONT = ImageFont.truetype("arial.ttf", FONT_SIZE)
except OSError:
    FONT = ImageFont.load_default()

LABEL_HEIGHT = FONT_SIZE + 10  # место под подпись

In [4]:
def find_triplets(folder: Path):
    """
    Ищет тройки *_lr.*, *_hr.*, *_pred.* в папке.
    Возвращает список словарей:
      [{'id': base, 'lr': Path, 'hr': Path, 'pred': Path}, ...]
    """
    exts = {".png", ".jpg", ".jpeg", ".tif", ".tiff", ".bmp", ".webp"}
    files = [p for p in folder.iterdir() if p.is_file() and p.suffix.lower() in exts]

    rx = re.compile(r"^(?P<base>.*?)[_-]?(?P<kind>lr|hr|pred)$", re.IGNORECASE)
    buckets = {}
    for f in files:
        m = rx.match(f.stem)
        if not m:
            continue
        base = m.group("base")
        kind = m.group("kind").lower()
        d = buckets.setdefault(base, {})
        d[kind] = f

    triplets = []
    for base, d in sorted(buckets.items()):
        if all(k in d for k in ("lr", "hr", "pred")):
            triplets.append({"id": base, "lr": d["lr"], "hr": d["hr"], "pred": d["pred"]})
    return triplets


def compute_center_box(w, h, box_w, box_h, dx=0, dy=0):
    cx, cy = w // 2 + dx, h // 2 + dy
    x0 = max(0, cx - box_w // 2)
    y0 = max(0, cy - box_h // 2)
    x1 = min(w, x0 + box_w)
    y1 = min(h, y0 + box_h)
    return (x0, y0, x1, y1)


def clamp_box(box, w, h):
    x0, y0, x1, y1 = box
    x0 = max(0, min(x0, w - 1))
    y0 = max(0, min(y0, h - 1))
    x1 = max(x0 + 1, min(x1, w))
    y1 = max(y0 + 1, min(y1, h))
    return (x0, y0, x1, y1)


def _text_size(draw: ImageDraw.ImageDraw, text: str, font: ImageFont.FreeTypeFont):
    # Pillow 10+: textbbox; старые Pillow: textsize
    try:
        bbox = draw.textbbox((0, 0), text, font=font)
        return bbox[2] - bbox[0], bbox[3] - bbox[1]
    except AttributeError:
        return draw.textsize(text, font=font)


def make_tile(img: Image.Image, label: str):
    """
    Делает тайл: увеличенное изображение + подпись снизу.
    """
    # увеличиваем изображение
    up_w, up_h = img.size[0] * UPSCALE_TILE, img.size[1] * UPSCALE_TILE
    up = img.resize((up_w, up_h), Image.NEAREST)

    tile_w = up_w + PADDING * 2
    tile_h = up_h + PADDING * 2 + LABEL_HEIGHT
    tile = Image.new("RGB", (tile_w, tile_h), (20, 20, 20))
    tile.paste(up, (PADDING, PADDING))

    draw = ImageDraw.Draw(tile)
    text = str(label)
    tw, th = _text_size(draw, text, FONT)
    tx = (tile_w - tw) // 2
    ty = up_h + PADDING + (LABEL_HEIGHT - th) // 2
    draw.text((tx, ty), text, fill=(240, 240, 240), font=FONT)
    return tile


def make_orig_tile(orig_img: Image.Image, box, crop_tile_height: int, label_for_kind: str = "HR"):
    """
    Делает тайл из «оригинального» HR с красной рамкой box.
    Высота предпросмотра подгоняется под высоту тайла кропа.
    """
    w, h = orig_img.size

    # подгоним высоту предпросмотра под высоту кроп-тайла (примерно)
    preview_h = crop_tile_height - LABEL_HEIGHT - 2 * PADDING
    scale = preview_h / h
    preview_w = int(w * scale)
    preview = orig_img.resize((preview_w, preview_h), Image.BICUBIC)

    # пересчитаем box в координаты уменьшенного предпросмотра
    sx = preview_w / w
    sy = preview_h / h
    x0, y0, x1, y1 = box
    rx0 = int(x0 * sx)
    ry0 = int(y0 * sy)
    rx1 = int(x1 * sx)
    ry1 = int(y1 * sy)

    tile_w = preview.width + 2 * PADDING
    tile_h = preview.height + 2 * PADDING + LABEL_HEIGHT
    tile = Image.new("RGB", (tile_w, tile_h), (20, 20, 20))
    tile.paste(preview, (PADDING, PADDING))

    draw = ImageDraw.Draw(tile)

    # рамка (немного утолщённая)
    for k in range(RECT_WIDTH):
        draw.rectangle(
            (PADDING + rx0 - k, PADDING + ry0 - k,
             PADDING + rx1 + k, PADDING + ry1 + k),
            outline=(220, 40, 40)
        )

    text = f"HR + BOX ({label_for_kind})"
    tw, th = _text_size(draw, text, FONT)
    tx = (tile_w - tw) // 2
    ty = preview.height + PADDING + (LABEL_HEIGHT - th) // 2
    draw.text((tx, ty), text, fill=(240, 240, 240), font=FONT)
    return tile


def hstack(tiles):
    w = sum(t.width for t in tiles) + GUTTER * (len(tiles) - 1)
    h = max(t.height for t in tiles)
    out = Image.new("RGB", (w, h), (10, 10, 10))
    x = 0
    for i, t in enumerate(tiles):
        out.paste(t, (x, 0))
        x += t.width + (GUTTER if i < len(tiles) - 1 else 0)
    return out


def vstack(rows):
    w = max(r.width for r in rows)
    h = sum(r.height for r in rows) + GUTTER * (len(rows) - 1)
    out = Image.new("RGB", (w, h), (5, 5, 5))
    y = 0
    for i, r in enumerate(rows):
        out.paste(r, ((w - r.width) // 2, y))
        y += r.height + (GUTTER if i < len(rows) - 1 else 0)
    return out


def get_crop_box_for_material(mat_name: str, hr_img: Image.Image):
    """
    Возвращает box в координатах HR для данной породы.
    Либо ABS_BOX, либо центр + GLOBAL_BOX_SIZE/OFFSET.
    """
    w, h = hr_img.size
    if USE_ABS_BOX.get(mat_name, USE_ABS_BOX["default"]):
        box = ABS_BOX.get(mat_name, ABS_BOX["default"])
    else:
        bw, bh = GLOBAL_BOX_SIZE
        dx, dy = GLOBAL_BOX_OFFSET
        box = compute_center_box(w, h, bw, bh, dx=dx, dy=dy)
    return clamp_box(box, w, h)


In [5]:
from PIL import ImageDraw

def add_box_overlay(img: Image.Image, box, label_for_kind: str | None = None):
    """
    Рисует красный квадрат на полном изображении (без ресайза).
    box — в координатах этого img.
    """
    img = img.convert("RGB").copy()
    draw = ImageDraw.Draw(img)

    x0, y0, x1, y1 = box
    for k in range(RECT_WIDTH):
        draw.rectangle(
            (x0 - k, y0 - k, x1 + k, y1 + k),
            outline=(220, 40, 40)
        )
    '''
    # Надпись (опционально)
    if label_for_kind is not None:
        text = f"HR + BOX ({label_for_kind})"
        tw, th = _text_size(draw, text, FONT)
        tx = (img.width - tw) // 2
        ty = img.height - th - 5
        draw.text((tx, ty), text, fill=(240, 240, 240), font=FONT)
    '''
    return img

def build_montage_for_material(mat_name: str, max_triplets: int | None = MAX_TRIPLETS_PER_MAT):
    mat_dir = ROOT / mat_name
    if not mat_dir.exists():
        print(f"[{mat_name}] нет директории: {mat_dir}")
        return None

    triplets = find_triplets(mat_dir)
    if not triplets:
        print(f"[{mat_name}] не найдено троек LR/HR/PRED")
        return None

    if max_triplets is not None:
        triplets = triplets[:max_triplets]

    rows = []

    for trip in triplets:
        base = trip["id"]

        # читаем в L
        lr = Image.open(trip["lr"]).convert("L")
        hr = Image.open(trip["hr"]).convert("L")
        pred = Image.open(trip["pred"]).convert("L")

        # целевой размер — размер HR (например, 500x500)
        target_size = hr.size  # (W, H)

        # масштаб между LR и HR
        w_lr, h_lr = lr.size
        w_hr, h_hr = hr.size
        sx = w_hr / w_lr
        sy = h_hr / h_lr

        # box в координатах HR
        box_hr = get_crop_box_for_material(mat_name, hr)

        # box для LR (масштабируем из HR)
        x0, y0, x1, y1 = box_hr
        lr_box = (
            int(x0 / sx),
            int(y0 / sy),
            int(x1 / sx),
            int(y1 / sy),
        )
        lr_box = clamp_box(lr_box, w_lr, h_lr)

        # === FULL-версии, приведённые к одному размеру ===
        lr_full = lr.resize(target_size, Image.BICUBIC)
        hr_full = hr  # уже target_size
        pred_full = pred.resize(target_size, Image.BICUBIC) if pred.size != target_size else pred

        # HR с красной рамкой (полный кадр)
        hr_with_box = add_box_overlay(hr_full, box_hr, label_for_kind=mat_name)

        # === КРОПЫ ===
        # вырезаем маленькие кусочки...
        lr_crop = lr.crop(lr_box)
        hr_crop = hr.crop(box_hr)
        pred_crop = pred.crop(box_hr)

        # ...и ресайзим их до target_size, чтобы всё было 500x500
        lr_crop_up = lr_crop.resize(target_size, Image.BICUBIC)
        hr_crop_up = hr_crop.resize(target_size, Image.BICUBIC)
        pred_crop_up = pred_crop.resize(target_size, Image.BICUBIC)

        # === собираем 7 тайлов (все с одинаковой базовой геометрией) ===
        lr_tile   = make_tile(lr_full,   f"{mat_name} LR")
        hr_tile   = make_tile(hr_full,   f"{mat_name} HR")
        pred_tile = make_tile(pred_full, f"{mat_name} PRED")
        hr_box_tile   = make_tile(hr_with_box,  "HR + BOX")

        lr_crop_tile   = make_tile(lr_crop_up,   "LR CROP")
        hr_crop_tile   = make_tile(hr_crop_up,   "HR CROP")
        pred_crop_tile = make_tile(pred_crop_up, "PRED CROP")

        row = hstack([
            lr_tile, hr_tile, pred_tile,
            hr_box_tile,
            lr_crop_tile, hr_crop_tile, pred_crop_tile
        ])
        rows.append(row)

    montage = vstack(rows)
    out_path = OUT_DIR / f"{mat_name}_montage_x2.png"
    montage.save(out_path)
    print(f"[{mat_name}] сохранён монтаж: {out_path}")
    return out_path

montage_paths = {}
for mat in MATERIALS:
    p = build_montage_for_material(mat)
    montage_paths[mat] = p
    '''
    if p is not None:
        display(Image.open(p))
    '''

[carbonate2D] сохранён монтаж: ..\..\deeprock_x2_report\carbonate2D_montage_x2.png
[sandstone2D] сохранён монтаж: ..\..\deeprock_x2_report\sandstone2D_montage_x2.png
[coal2D] сохранён монтаж: ..\..\deeprock_x2_report\coal2D_montage_x2.png


In [6]:
def collect_pixels_for_material(mat_name: str, max_triplets: int | None = MAX_TRIPLETS_PER_MAT):
    """
    Собирает значения пикселей для:
      - FULL: все пиксели LR/HR/PRED
      - CROP: только внутри бокса (из шага 1)
    Возвращает два словаря по ключам 'lr', 'hr', 'pred'.
    """
    mat_dir = ROOT / mat_name
    triplets = find_triplets(mat_dir)
    if not triplets:
        return None, None

    if max_triplets is not None:
        triplets = triplets[:max_triplets]

    full = {"lr": [], "hr": [], "pred": []}
    crop = {"lr": [], "hr": [], "pred": []}

    for trip in triplets:
        lr = Image.open(trip["lr"]).convert("L")
        hr = Image.open(trip["hr"]).convert("L")
        pred = Image.open(trip["pred"]).convert("L")

        # FULL
        full["lr"].append(np.array(lr, dtype=np.float32).ravel())
        full["hr"].append(np.array(hr, dtype=np.float32).ravel())
        full["pred"].append(np.array(pred, dtype=np.float32).ravel())

        # CROP
        box_hr = get_crop_box_for_material(mat_name, hr)
        w_lr, h_lr = lr.size
        w_hr, h_hr = hr.size
        sx = w_hr / w_lr
        sy = h_hr / h_lr
        x0, y0, x1, y1 = box_hr
        lr_box = (
            int(x0 / sx),
            int(y0 / sy),
            int(x1 / sx),
            int(y1 / sy),
        )
        lr_box = clamp_box(lr_box, w_lr, h_lr)

        lr_crop = lr.crop(lr_box)
        hr_crop = hr.crop(box_hr)
        pred_crop = pred.crop(box_hr)

        crop["lr"].append(np.array(lr_crop, dtype=np.float32).ravel())
        crop["hr"].append(np.array(hr_crop, dtype=np.float32).ravel())
        crop["pred"].append(np.array(pred_crop, dtype=np.float32).ravel())

    # конкатенируем
    for dic in (full, crop):
        for k in dic:
            if dic[k]:
                dic[k] = np.concatenate(dic[k], axis=0)
            else:
                dic[k] = np.array([], dtype=np.float32)

    return full, crop


def plot_overlapped_histograms(data_dict: dict, title: str, out_path: Path | None = None,
                               bins: int = 256, density: bool = True):
    """
    Рисует наложенные гистограммы для 'lr', 'hr', 'pred'.
    data_dict: {'lr': np.ndarray, 'hr': ..., 'pred': ...}
    """
    plt.figure(figsize=(8, 5))
    for key, color in zip(["lr", "hr", "pred"], ["tab:blue", "tab:orange", "tab:green"]):
        arr = data_dict[key]
        if arr.size == 0:
            continue
        plt.hist(arr, bins=bins, density=density, alpha=0.5, label=key.upper(), color=color)
    plt.xlabel("Intensity (0-255)")
    plt.ylabel("Density" if density else "Count")
    plt.title(title)
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    if out_path is not None:
        plt.savefig(out_path, dpi=150)
        print(f"Сохранён график: {out_path}")
    plt.show()


def make_histograms_for_material(mat_name: str, max_triplets: int | None = MAX_TRIPLETS_PER_MAT):
    full, crop = collect_pixels_for_material(mat_name, max_triplets=max_triplets)
    if full is None:
        print(f"[{mat_name}] нет данных для гистограмм")
        return

    # FULL
    out_full = OUT_DIR / f"{mat_name}_hist_full.png"
    plot_overlapped_histograms(
        full,
        title=f"{mat_name}: распределения интенсивностей (FULL, x2)",
        out_path=out_full,
        bins=256,
        density=True,
    )

    # CROP
    out_crop = OUT_DIR / f"{mat_name}_hist_crop.png"
    plot_overlapped_histograms(
        crop,
        title=f"{mat_name}: распределения интенсивностей (CROP, x2)",
        out_path=out_crop,
        bins=256,
        density=True,
    )


# Запустить для всех пород
'''
for mat in MATERIALS:
    make_histograms_for_material(mat)
'''

'\nfor mat in MATERIALS:\n    make_histograms_for_material(mat)\n'

In [7]:
from IPython.display import display

def normalize_diff_to_uint8(diff_arr: np.ndarray) -> Image.Image:
    """
    Преобразует массив разностей в uint8 [0,255] для визуализации.
    Берём модуль, нормируем на [0, 1].
    """
    diff_abs = np.abs(diff_arr)
    if diff_abs.max() > 0:
        diff_norm = diff_abs / diff_abs.max()
    else:
        diff_norm = diff_abs
    img = (diff_norm * 255).astype(np.uint8)
    return Image.fromarray(img, "L")


def make_diff_images_for_material(
    mat_name: str,
    max_triplets: int | None = MAX_TRIPLETS_PER_MAT,
    show: bool = True,
):
    mat_dir = ROOT / mat_name
    triplets = find_triplets(mat_dir)
    if not triplets:
        print(f"[{mat_name}] нет троек для разностей")
        return

    if max_triplets is not None:
        triplets = triplets[:max_triplets]

    out_dir = OUT_DIR / f"{mat_name}_diffs"
    out_dir.mkdir(parents=True, exist_ok=True)

    for trip in triplets:
        base = trip["id"]

        lr = Image.open(trip["lr"]).convert("L")
        hr = Image.open(trip["hr"]).convert("L")
        pred = Image.open(trip["pred"]).convert("L")

        # приводим LR к размеру HR
        lr_up = lr.resize(hr.size, Image.BICUBIC)

        hr_arr = np.array(hr, dtype=np.float32)
        pred_arr = np.array(pred, dtype=np.float32)
        lr_arr = np.array(lr_up, dtype=np.float32)

        # разности
        diff_hr_lr = hr_arr - lr_arr
        diff_pred_lr = pred_arr - lr_arr

        img_diff_hr_lr = normalize_diff_to_uint8(diff_hr_lr)
        img_diff_pred_lr = normalize_diff_to_uint8(diff_pred_lr)

        # коллаж: HR, LR↑, |HR-LR| / PRED, LR↑, |PRED-LR|
        strip1 = hstack([
            make_tile(hr,      f"{mat_name} HR"),
            make_tile(lr_up,   f"{mat_name} LR"),
            make_tile(img_diff_hr_lr, "HR - LR"),
        ])
        #lr_tile   = make_tile(lr_full,   f"{mat_name} LR")
        #hr_tile   = make_tile(hr_full,   f"{mat_name} HR")
        #pred_tile = make_tile(pred_full, f"{mat_name} PRED")

        strip2 = hstack([
            make_tile(pred,            f"{mat_name} PRED"),
            make_tile(lr_up,           f"{mat_name} LR"),
            make_tile(img_diff_pred_lr, "PRED - LR"),
        ])

        combined = vstack([strip1, strip2])

        out_path = out_dir / f"{base}_diffs.png"
        combined.save(out_path)

        if show:
            print(f"[{mat_name}] {base} | разности (HR-LR, PRED-LR)")
            display(combined)

    print(f"[{mat_name}] сохранены дифф-картины в {out_dir}")


# Генерируем разности для всех пород и сразу показываем
'''
for mat in MATERIALS:
    make_diff_images_for_material(mat, show=True)
'''

'\nfor mat in MATERIALS:\n    make_diff_images_for_material(mat, show=True)\n'

In [17]:
from pathlib import Path
from typing import Optional, Tuple

import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

# --------- Настройки по умолчанию ---------
DEFAULT_OUT_DIR = Path("sr_report_generic")
DEFAULT_OUT_DIR.mkdir(exist_ok=True, parents=True)

# Размер центрального кропа по умолчанию, если crop_box не задан
DEFAULT_CROP_SIZE = (100, 100)  # (width, height)

# --------- Вспомогательные функции ---------
from PIL import ImageDraw, ImageFont

def _text_size(draw: ImageDraw.ImageDraw, text: str, font: ImageFont.ImageFont):
    """
    Универсальная функция измерения текста для разных версий Pillow.
    Возвращает (width, height).
    """
    # Новый способ — через textbbox (есть в современных Pillow)
    if hasattr(draw, "textbbox"):
        left, top, right, bottom = draw.textbbox((0, 0), text, font=font)
        return right - left, bottom - top

    # Запасной вариант — через методы шрифта
    if hasattr(font, "getbbox"):
        left, top, right, bottom = font.getbbox(text)
        return right - left, bottom - top

    # Совсем старый запасной вариант
    return font.getsize(text)

def load_triplet(
    lr_path: Path,
    hr_path: Path,
    sr_path: Path,
) -> Tuple[Image.Image, Image.Image, Image.Image]:
    """Загрузить LR/HR/SR как grayscale, привести LR и SR к размеру HR."""
    lr = Image.open(lr_path).convert("L")
    hr = Image.open(hr_path).convert("L")
    sr = Image.open(sr_path).convert("L")

    target_size = hr.size
    lr_up = lr.resize(target_size, Image.BICUBIC)
    if sr.size != target_size:
        sr_full = sr.resize(target_size, Image.BICUBIC)
    else:
        sr_full = sr

    return lr_up, hr, sr_full


def normalize_diff_to_uint8(a: Image.Image, b: Image.Image) -> Image.Image:
    """Абсолютная разность |a-b|, нормированная в диапазон [0,255]."""
    arr_a = np.asarray(a, dtype=np.float32)
    arr_b = np.asarray(b, dtype=np.float32)
    diff_abs = np.abs(arr_a - arr_b)
    maxv = diff_abs.max()
    if maxv > 0:
        diff_norm = diff_abs / maxv
    else:
        diff_norm = diff_abs
    img = (diff_norm * 255).astype(np.uint8)
    return Image.fromarray(img, "L")


def otsu_threshold(arr_uint8: np.ndarray) -> int:
    """Порог Отсу для uint8-изображения."""
    hist, _ = np.histogram(arr_uint8.ravel(), bins=256, range=(0, 255))
    total = arr_uint8.size
    sum_total = np.dot(np.arange(256), hist)

    sum_b = 0.0
    w_b = 0
    max_var = 0.0
    thresh = 0

    for t in range(256):
        w_b += hist[t]
        if w_b == 0:
            continue
        w_f = total - w_b
        if w_f == 0:
            break
        sum_b += t * hist[t]
        m_b = sum_b / w_b
        m_f = (sum_total - sum_b) / w_f
        var_between = w_b * w_f * (m_b - m_f) ** 2
        if var_between > max_var:
            max_var = var_between
            thresh = t
    return thresh


def binarize_from_hr_threshold(
    img_hr: Image.Image,
    img_lr_up: Image.Image,
    img_sr: Image.Image,
):
    """Бинаризация HR/LR_up/SR с единым порогом (Отсу по HR)."""
    hr_np = np.asarray(img_hr, dtype=np.uint8)
    lr_np = np.asarray(img_lr_up, dtype=np.uint8)
    sr_np = np.asarray(img_sr, dtype=np.uint8)

    thr = otsu_threshold(hr_np)

    def _bin(a):
        m = (a > thr).astype("uint8") * 255
        return Image.fromarray(m, mode="L")

    return _bin(hr_np), _bin(lr_np), _bin(sr_np), thr


def xor_mask(a: Image.Image, b: Image.Image) -> Image.Image:
    """XOR двух бинарных масок (0/255) -> 0/255."""
    a_np = np.asarray(a, dtype=np.uint8) > 0
    b_np = np.asarray(b, dtype=np.uint8) > 0
    xor_np = (a_np ^ b_np).astype("uint8") * 255
    return Image.fromarray(xor_np, mode="L")

from PIL import ImageDraw, ImageFont

def add_label(img: Image.Image, text: str, font_size: int | None = None) -> Image.Image:
    """
    Добавляет подпись внизу изображения.

    Если font_size=None, выбирает размер шрифта автоматически из высоты картинки:
      - для больших full-изображений шрифт крупнее,
      - для маленьких кропов шрифт меньше.
    """
    w, h = img.size

    # Автоматический размер шрифта ~6% от высоты, c ограничениями
    if font_size is None:
        font_size = int(h * 0.06)          # коэффициент можно подрегулировать
        font_size = max(10, min(28, font_size))

    pad = 4
    label_height = font_size + 2 * pad

    canvas = Image.new("RGB", (w, h + label_height), (20, 20, 20))
    canvas.paste(img.convert("RGB"), (0, 0))

    draw = ImageDraw.Draw(canvas)
    try:
        font = ImageFont.truetype("arial.ttf", font_size)
    except OSError:
        font = ImageFont.load_default()

    tw, th = _text_size(draw, text, font)
    tx = (w - tw) // 2
    ty = h + pad
    draw.text((tx, ty), text, fill=(240, 240, 240), font=font)

    return canvas


def hstack(images):
    """Горизонтальная склейка списка PIL.Image."""
    widths, heights = zip(*(im.size for im in images))
    total_w = sum(widths)
    max_h = max(heights)
    out = Image.new("RGB", (total_w, max_h), (0, 0, 0))
    x = 0
    for im in images:
        out.paste(im, (x, 0))
        x += im.size[0]
    return out


def vstack(images):
    """Вертикальная склейка списка PIL.Image."""
    widths, heights = zip(*(im.size for im in images))
    max_w = max(widths)
    total_h = sum(heights)
    out = Image.new("RGB", (max_w, total_h), (0, 0, 0))
    y = 0
    for im in images:
        out.paste(im, (0, y))
        y += im.size[1]
    return out


def get_center_crop_box(
    img: Image.Image,
    crop_size: Tuple[int, int] = DEFAULT_CROP_SIZE,
) -> Tuple[int, int, int, int]:
    """Центральный кроп заданного размера (если не хочешь руками считать box)."""
    W, H = img.size
    cw, ch = crop_size
    cw = min(cw, W)
    ch = min(ch, H)
    left = (W - cw) // 2
    upper = (H - ch) // 2
    right = left + cw
    lower = upper + ch
    return (left, upper, right, lower)


def plot_hists(hr, lr_up, sr, hr_c, lr_c, sr_c, out_path):
    """
    Гистограммы интенсивностей:
      слева  – full (HR, LR, SR наложены),
      справа – crop (HR, LR, SR наложены).
    У каждой оси своя шкала по count, поэтому crop не «сплющивается».
    """
    hr_np = np.asarray(hr, dtype=np.uint8).ravel()
    lr_np = np.asarray(lr_up, dtype=np.uint8).ravel()
    sr_np = np.asarray(sr, dtype=np.uint8).ravel()

    hr_c_np = np.asarray(hr_c, dtype=np.uint8).ravel()
    lr_c_np = np.asarray(lr_c, dtype=np.uint8).ravel()
    sr_c_np = np.asarray(sr_c, dtype=np.uint8).ravel()

    bins = np.linspace(0, 255, 256)

    fig, axes = plt.subplots(1, 2, figsize=(10, 4), sharex=True, sharey=False)

    ax_full, ax_crop = axes

    # --- FULL ---
    ax_full.hist(hr_np, bins=bins, histtype="step", linewidth=1.5, label="HR")
    ax_full.hist(lr_np, bins=bins, histtype="step", linewidth=1.5, label="LR")
    ax_full.hist(sr_np, bins=bins, histtype="step", linewidth=1.5, label="SR")
    ax_full.set_title("Full image")
    ax_full.set_xlabel("Intensity")
    ax_full.set_ylabel("Count")
    ax_full.legend()

    # --- CROP ---
    ax_crop.hist(hr_c_np, bins=bins, histtype="step", linewidth=1.5, label="HR crop")
    ax_crop.hist(lr_c_np, bins=bins, histtype="step", linewidth=1.5, label="LR crop")
    ax_crop.hist(sr_c_np, bins=bins, histtype="step", linewidth=1.5, label="SR crop")
    ax_crop.set_title("Crop")
    ax_crop.set_xlabel("Intensity")
    ax_crop.set_ylabel("Count")
    ax_crop.legend()

    plt.tight_layout()
    fig.savefig(out_path, dpi=200)
    plt.close(fig)


# --------- Главная функция: сделать 5 фигур ---------
def build_5_figures_for_triplet(
    lr_path: str,
    hr_path: str,
    sr_path: str,
    out_dir: Path = DEFAULT_OUT_DIR,
    base_name: Optional[str] = None,
    crop_box: Optional[Tuple[int, int, int, int]] = None,
):
    """
    Строит 5 фигур для одного триплета LR/HR/SR, без привязки к породам:

      01_intensity_full: HR/LR/SR и разности
      02_binary_full: бинаризация + XOR
      03_intensity_crop: то же, но для кропа
      04_binary_crop: бинаризация + XOR для кропа
      05_histograms: гистограммы full + crop

    lr_path, hr_path, sr_path: пути к изображениям (любого датасета).
    crop_box: (left, upper, right, lower); если None — берётся центральный кроп.
    """
    lr_path = Path(lr_path)
    hr_path = Path(hr_path)
    sr_path = Path(sr_path)
    out_dir.mkdir(exist_ok=True, parents=True)

    if base_name is None:
        # какое-то разумное имя по умолчанию
        base_name = hr_path.stem

    # --- загрузка и приведение к одному размеру ---
    lr_up, hr, sr_full = load_triplet(lr_path, hr_path, sr_path)

    # ========= 1. FULL: интенсивности и разности =========
    diff_hr_lr = normalize_diff_to_uint8(hr, lr_up)
    diff_hr_sr = normalize_diff_to_uint8(hr, sr_full)
    diff_sr_lr = normalize_diff_to_uint8(sr_full, lr_up)

    row1 = hstack([
        add_label(hr,      "HR"),
        add_label(lr_up,   "LR"),
        add_label(diff_hr_lr, "|HR-LR|"),
    ])
    row2 = hstack([
        add_label(hr,      "HR"),
        add_label(sr_full, "SR"),
        add_label(diff_hr_sr, "|HR-SR|"),
    ])
    row3 = hstack([
        add_label(diff_sr_lr, "|SR-LR|"),
    ])

    fig1 = vstack([row1, row2, row3])
    fig1_path = out_dir / f"{base_name}_01_intensity_full.png"
    fig1.save(fig1_path)

    # ========= 2. FULL: бинаризация и XOR =========
    bin_hr, bin_lr, bin_sr, thr = binarize_from_hr_threshold(hr, lr_up, sr_full)
    xor_hr_lr = xor_mask(bin_hr, bin_lr)
    xor_hr_sr = xor_mask(bin_hr, bin_sr)
    xor_sr_lr = xor_mask(bin_sr, bin_lr)

    row1_b = hstack([
        add_label(bin_hr, f"HR bin (thr={thr})"),
        add_label(bin_lr, "LR bin"),
        add_label(xor_hr_lr, "HR xor LR"),
    ])
    row2_b = hstack([
        add_label(bin_hr, "HR bin"),
        add_label(bin_sr, "SR bin"),
        add_label(xor_hr_sr, "HR xor SR"),
    ])
    row3_b = hstack([
        add_label(xor_sr_lr, "SR xor LR"),
    ])

    fig2 = vstack([row1_b, row2_b, row3_b])
    fig2_path = out_dir / f"{base_name}_02_binary_full.png"
    fig2.save(fig2_path)

    # ========= 3–4. CROP =========
    if crop_box is None:
        crop_box = get_center_crop_box(hr, DEFAULT_CROP_SIZE)

    hr_c = hr.crop(crop_box)
    lr_c = lr_up.crop(crop_box)
    sr_c = sr_full.crop(crop_box)

    diff_hr_lr_c = normalize_diff_to_uint8(hr_c, lr_c)
    diff_hr_sr_c = normalize_diff_to_uint8(hr_c, sr_c)
    diff_sr_lr_c = normalize_diff_to_uint8(sr_c, lr_c)

    row1_c = hstack([
        add_label(hr_c, "HR crop"),
        add_label(lr_c, "LR crop"),
        add_label(diff_hr_lr_c, "|HR-LR| crop"),
    ])
    row2_c = hstack([
        add_label(hr_c, "HR crop"),
        add_label(sr_c, "SR crop"),
        add_label(diff_hr_sr_c, "|HR-SR| crop"),
    ])
    row3_c = hstack([
        add_label(diff_sr_lr_c, "|SR-LR| crop"),
    ])

    fig3 = vstack([row1_c, row2_c, row3_c])
    fig3_path = out_dir / f"{base_name}_03_intensity_crop.png"
    fig3.save(fig3_path)

    # бинаризация кропов
    bin_hr_c, bin_lr_c, bin_sr_c, thr_c = binarize_from_hr_threshold(hr_c, lr_c, sr_c)
    xor_hr_lr_c = xor_mask(bin_hr_c, bin_lr_c)
    xor_hr_sr_c = xor_mask(bin_hr_c, bin_sr_c)
    xor_sr_lr_c = xor_mask(bin_sr_c, bin_lr_c)

    row1_bc = hstack([
        add_label(bin_hr_c, f"HR bin crop (thr={thr_c})"),
        add_label(bin_lr_c, "LR bin crop"),
        add_label(xor_hr_lr_c, "HR xor LR crop"),
    ])
    row2_bc = hstack([
        add_label(bin_hr_c, "HR bin crop"),
        add_label(bin_sr_c, "SR bin crop"),
        add_label(xor_hr_sr_c, "HR xor SR crop"),
    ])
    row3_bc = hstack([
        add_label(xor_sr_lr_c, "SR xor LR crop"),
    ])

    fig4 = vstack([row1_bc, row2_bc, row3_bc])
    fig4_path = out_dir / f"{base_name}_04_binary_crop.png"
    fig4.save(fig4_path)

    # ========= 5. ГИСТОГРАММЫ =========
    fig5_path = out_dir / f"{base_name}_05_histograms.png"
    plot_hists(hr, lr_up, sr_full, hr_c, lr_c, sr_c, fig5_path)

    print("Сохранены фигуры:")
    for p in [fig1_path, fig2_path, fig3_path, fig4_path, fig5_path]:
        print(" ", p)

In [23]:
build_5_figures_for_triplet(
    lr_path="C:/Users/Вячеслав/Documents/superresolution/modules/preds_x2/sample_5_lr.png",
    hr_path="C:/Users/Вячеслав/Documents/superresolution/modules/preds_x2/sample_5_hr.png",
    sr_path="C:/Users/Вячеслав/Documents/superresolution/modules/preds_x2/sample_5_pred.png",
    out_dir=Path("my_report/sample_5"),
    base_name="sample5",              # опционально
    crop_box=None                      # или (x1, y1, x2, y2)
)

C:\Users\Вячеслав\AppData\Local\Temp\ipykernel_21724\2704670222.py:67: DeprecationWarning: 'mode' parameter is deprecated and will be removed in Pillow 13 (2026-10-15)
  return Image.fromarray(img, "L")
C:\Users\Вячеслав\AppData\Local\Temp\ipykernel_21724\2704670222.py:112: DeprecationWarning: 'mode' parameter is deprecated and will be removed in Pillow 13 (2026-10-15)
  return Image.fromarray(m, mode="L")
C:\Users\Вячеслав\AppData\Local\Temp\ipykernel_21724\2704670222.py:122: DeprecationWarning: 'mode' parameter is deprecated and will be removed in Pillow 13 (2026-10-15)
  return Image.fromarray(xor_np, mode="L")


Сохранены фигуры:
  my_report\sample_5\sample5_01_intensity_full.png
  my_report\sample_5\sample5_02_binary_full.png
  my_report\sample_5\sample5_03_intensity_crop.png
  my_report\sample_5\sample5_04_binary_crop.png
  my_report\sample_5\sample5_05_histograms.png
